### **Attention is a technique that lets a model focus on the most relevant parts of the input when producing an output — instead of treating everything as equally important.**

- Think of it like how humans read:
When you read a sentence, you don’t remember every word equally — your brain pays more attention to the important words.

 Simple Example (Human-style):

Sentence: “The cat sat on the mat because it was tired.”

Question: What does “it” refer to?

Your brain instantly links “it” → “cat”, not “mat”.

### **Self-attention means:**

- Words attend to other words in the same sentence

- Every word understands its context

*Example:*

“I went to the bank to deposit money”

The word “bank” attends strongly to “deposit” and “money”, not “river”

### **Types of attention mechanisms:-** 
1. Simplified Self-Attention (Each word decides which other words are important)

2. Self-Attention (Each word uses query, key, and value to understand context from all words)

3. Causal Attention (Each word can attend only to past words, not future ones)

4. Multi-Head Attention (Multiple attention heads look at the same sentence from different perspectives).